In [6]:
import os, json, traceback
import pandas as pd

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    openai_api_key = KEY,
    model_name = 'gpt-4o-mini',
    temperature = 0.5
)